In [2]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 199 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=dac4a57dec2c5c1cb0aaeea56c79086195ee162d7df956fd13fcff08e4cb9666
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
#load data by sparksession

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('diabetes').getOrCreate()
df = spark.read.csv('drive/My Drive/Colab Notebooks/Dataset/diabetes.csv', header = True, inferSchema = True)
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [4]:
#Check type and column
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [5]:
#Mix all feature exclude label
df2 = df.drop('Outcome')
df2.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [6]:
#Data cleaning
#Delete missing values
df = df.replace('null', None).dropna(how='any')

In [7]:
#Create features columns for RF ML

from pyspark.ml.feature import VectorAssembler
selected = df2.columns
va = VectorAssembler(inputCols=selected, outputCol='features')
df_transform = va.transform(df)

#Train-test split
(train_data, test_data) = df_transform.randomSplit([0.8,0.2])

#Create model and fit
from pyspark.ml.classification import RandomForestClassifier


rf = RandomForestClassifier(labelCol='Outcome',
                        featuresCol='features',
                        maxDepth=5)


model = rf.fit(train_data)


In [10]:
#Evaluation
target_predictions = model.transform(test_data)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval = MulticlassClassificationEvaluator(labelCol = 'Outcome', metricName = 'accuracy')
print('Random Forest classifier Accuracy:', eval.evaluate(target_predictions))

Random Forest classifier Accuracy: 0.7414965986394558
